In [2]:
import cv2
import os
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.model_selection import train_test_split
import bm3d
from skimage import io, img_as_float
import matplotlib.pyplot as plt

# for denoising:
from skimage.restoration import denoise_wavelet, estimate_sigma
from skimage import data, img_as_float
from skimage.util import random_noise
from skimage.metrics import peak_signal_noise_ratio



work onn VISION dataset:
work on DO1 camera only

CALCULATING THE PRNU FOR A SPECIFIC CAMERA:
1. get all flat images in one folder - Sali
2. convert flat images to grayscale - Sali
3. denoising (using wavelet) - Aditya 
4. calculate noise residual (original - denoise image) for each image in the flat images - Aditya
5. calculate: equation in image "eq1' - Sali
6. calculate CRLB  in image 'eq2' - Sali

7. calculate linear pattern in image 'eq3', this is K which is the PRNU - write the equation in code format - Kavan
8. calculate conversion from RGB to grayscale in image 'eq4' or explore other conversion methods - Kavan

CALCULATING THE SIMILARITY BETWEEN A CAMERA'S FINGERPRINT AND A NEW TEST IMAGE:
work with natural images for camera DO1
9. calculate the noise residual (original image - denoised image)  (W2) noise residual for test image - Sri

#########################################

10. calculate X and Y in equations in img 'eq5'
11. Calculate the NCC in img 'eq6'
12. calculate the PCE in img 'eq7'



In [2]:
data_dir = '/data/Sali/camera_fingerprint/Dresden_Exp'
camera_ls = os.listdir(data_dir)
camera_ls = camera_ls[:1]

# Create list of all camera directories
camera_dirs = [data_dir + '/' + camera for camera in camera_ls ]
camera_dirs


['/data/Sali/camera_fingerprint/Dresden_Exp/Agfa_DC-504_0']

### Moving images to folder

In [ ]:
images_output_dir = '/data/Sali/camera_fingerprint/Vision_dataset_organized'

In [10]:
import shutil

In [3]:
# def move_images_to_folder(camera_dir, output_dir):
#     list_of_cameras = [f'D{str(i).zfill(2)}' for i in range(1, 37)]
#     subdirs = ['flat', 'nat', 'natFBH', 'natFBL', 'natWA']

#     for cam in list_of_cameras:
        
#         cam_dir = os.path.join(output_dir, cam)
#         os.makedirs(cam_dir, exist_ok=True)  
#         subdirs = ['flat', 'nat', 'natFBH', 'natFBL', 'natWA']

#         for sub in subdirs:
#             sub_dir = os.path.join(cam_dir, sub)
#             os.makedirs(sub_dir, exist_ok=True)

#     for image in os.listdir(camera_dir):
#         if image.endswith('.jpg') or image.endswith('.png'):
#             image_parts = image.split('_')

#             camera_id = image_parts[0]
#             subdir = image_parts[2]
#             current_img_path = os.path.join(camera_dir, image)
#             new_img_path = os.path.join(output_dir, camera_id, subdir, image)

#             try:
#                 shutil.copy(current_img_path, new_img_path)
#             except FileNotFoundError:
#                 print(f"Error: {current_img_path} not found.")
#             except Exception as e:
#                 print(f"Error copying {image}: {e}")
    
# move_images_to_folder('/data/Sali/camera_fingerprint/', images_output_dir)

### Extracting PRNU

In [ ]:
def extract_noise_residual(img):
    blur = cv2.GaussianBlur(img, (5,5), 0) #use filter in paper 

    noise_residual = img  - blur

    return noise_residual


In [ ]:
def extract_noise_residual(img):
    # wavelet denoising
    blur = cv2.GaussianBlur(img, (5,5), 0) #use filter in paper 

    noise_residual = img  - blur

    return noise_residual


In [ ]:
# def extract_noise_residual(img_path):
    
#     print(f'processing img: {img_path}\n')
#     sigma = 30/255

#     noisy_image = img_as_float(io.imread(img_path))
#     denoised_image = bm3d.bm3d(noisy_image, sigma_psd=sigma)

#     noise_residual = noisy_image  - denoised_image

#     return noise_residual


In [ ]:
def calculate_CRP(img_paths, camera_dir):
    residuals = []
    for img_path in img_paths:
        train_img_path = os.path.join(camera_dir, img_path)
        img = cv2.imread(train_img_path).astype(np.float32)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        noise_residual = extract_noise_residual(img)
        residuals.append(noise_residual)

    if residuals:

        average_residual = np.mean(residuals, axis=0) # change this to match the paper's method 
        return average_residual
    else:
        return None

    

In [ ]:
# def calculate_CRP(train_paths, camera_dir):
#     residuals = []
#     train_img_paths = [os.path.join(camera_dir, fname) for fname in train_paths]

#     with Pool(cpu_count()) as pool:
#         residuals = pool.map(extract_noise_residual, train_img_paths)

#     residuals = [res for res in residuals if res is not None]

#     if residuals:
#         average_residual = np.mean(residuals, axis=0)

#         return average_residual
    
#     else:
#         return None



In [ ]:
def Cross_Correlation(camera_CRP, test_img_PRNU):
    numerator = np.sum(camera_CRP * test_img_PRNU)
    denominator = np.sqrt(np.sum(camera_CRP**2) * np.sum(test_img_PRNU**2))
    return numerator/denominator


In [ ]:
for camera_dir in camera_dirs:
    img_paths = os.listdir(camera_dir)[:50]
    # train_paths, test_paths = train_test_split(img_paths , test_size=0.3, random_state=42)
    CRP = calculate_CRP(img_paths, camera_dir)

    if CRP is None:
        print('no valid training images')
        continue

    # for test_img in test_paths:
    #     test_img_path = os.path.join(camera_dir, test_img)
    #     img = cv2.imread(test_img_path).astype(np.float32)
    #     if img is None:
    #         continue

        
    #     test_residual = extract_noise_residual(img)
    #     similarity = Cross_Correlation(CRP, test_residual)
        
    #     print(f'{test_img}, Similarity: {similarity:.4f}')


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (35,) + inhomogeneous part.

In [ ]:
def crosscorr_2d(k1: np.ndarray, k2: np.ndarray) -> np.ndarray:
    """
    PRNU 2D cross-correlation
    :param k1: 2D matrix of size (h1,w1)
    :param k2: 2D matrix of size (h2,w2)
    :return: 2D matrix of size (max(h1,h2),max(w1,w2))
    """
    assert (k1.ndim == 2)
    assert (k2.ndim == 2)

    max_height = max(k1.shape[0], k2.shape[0])
    max_width = max(k1.shape[1], k2.shape[1])

    k1 -= k1.flatten().mean()
    k2 -= k2.flatten().mean()

    k1 = np.pad(k1, [(0, max_height - k1.shape[0]), (0, max_width - k1.shape[1])], mode='constant', constant_values=0)
    k2 = np.pad(k2, [(0, max_height - k2.shape[0]), (0, max_width - k2.shape[1])], mode='constant', constant_values=0)

    k1_fft = fft2(k1, )
    k2_fft = fft2(np.rot90(k2, 2), )

    return np.real(ifft2(k1_fft * k2_fft)).astype(np.float32)


In [ ]:
def pce(cc: np.ndarray, gt: np.ndarray, ) -> dict:
    """
    Compute statistics
    :param cc: cross-correlation or normalized cross-correlation matrix
    :param gt: boolean multidimensional array representing groundtruth
    :return: statistics dictionary
    """
    assert (cc.shape == gt.shape)
    assert (gt.dtype == np.bool)

    assert (cc.shape == gt.shape)
    assert (gt.dtype == np.bool)

    fpr, tpr, th = roc_curve(gt.flatten(), cc.flatten())
    auc_score = auc(fpr, tpr)

    # EER
    eer_idx = np.argmin((fpr - (1 - tpr)) ** 2, axis=0)
    eer = float(fpr[eer_idx])

    outdict = {
        'tpr': tpr,
        'fpr': fpr,
        'th': th,
        'auc': auc_score,
        'eer': eer,
    }

    return outdict


In [ ]:
def remove_fixed_pattern_noise(K_hat: np.ndarray) -> np.ndarray:
    """
    Removes fixed pattern noise (row and column) from a 2D matrix.
    
    Equation (eq3.png) implementation:
    1. r_i = 1/n * sum_{j=1}^{n} K_hat[i,j]
    2. K'[i,j] = K_hat[i,j] - r_i
    3. c_j = 1/m * sum_{i=1}^{m} K'[i,j]
    4. K''[i,j] = K'[i,j] - c_j

    """

    # Getting the dimensions of the input matrix K_hat
    m, n = K_hat.shape

    K_prime = np.copy(K_hat)
    K_double_prime = np.copy(K_prime)

    # r_i = 1/n * sum_{j=1}^{n} K_hat[i,j]
    
    r_i = np.mean(K_hat, axis=1)

    # Subtracting r_i from each row of K_hat to get K_prime.
    # K'[i,j] = K_hat[i,j] - r_i

    K_prime = K_hat - r_i[:, np.newaxis]

    #Finding the mean of each column in K_prime
    # c_j = 1/m * sum_{i=1}^{m} K'[i,j]
    
    c_j = np.mean(K_prime, axis=0)

    # Subtract c_j from each column of K_prime to get K_double_prime.
    # K''[i,j] = K'[i,j] - c_j
    
    K_double_prime = K_prime - c_j

    return K_double_prime

In [ ]:
https://github.com/samuelebortolotti/neural-prnu-extractor #code to replicate
https://github.com/ocrim1996/prnu-python?tab=readme-ov-file
https://ieeexplore.ieee.org/document/7791195 # image alignment 
https://ieeexplore.ieee.org/document/7791195 # paper that talks about denoising filter
https://paperswithcode.com/sota/image-denoising-on-sidd # papers with code
https://www.reddit.com/r/AirlinerAbduction2014/comments/1fnxy5o/photo_response_nonuniformity_prnu_authentication/#:~:text=Extracting%20the%20PRNU%20requires%20denoising,is%20compared%20to%20the%20CRP. # reddit page

SyntaxError: invalid syntax (4290831171.py, line 1)